<a href="https://colab.research.google.com/github/bird-feeder/BirdFSD-YOLOv5/blob/main/notebooks/BirdFSD_YOLOv5_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

**`Runtime` -->  `Change runtime type` --> `GPU`**

- Drag and drop your `.env` (or `secrets.txt`) file to the files section of this notebook.

In [ ]:
! wget -q 'https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh'
! bash "Miniforge3-Linux-x86_64.sh" -b -p

In [ ]:
! git clone https://github.com/bird-feeder/BirdFSD-YOLOv5.git

import shutil
from pathlib import Path
if Path('.env').exists():
    shutil.move('.env', 'BirdFSD-YOLOv5/.env')
elif Path('secrets.txt').exists():
    shutil.move('secrets.txt', 'BirdFSD-YOLOv5/.env')

%cd /content/BirdFSD-YOLOv5

! /root/miniforge3/bin/python -m pip install -e .
! /root/miniforge3/bin/pip install -r requirements.txt

! git clone https://github.com/ultralytics/yolov5.git

In [ ]:
#@title Login to W&B
! wandb login

## Download model weights

In [ ]:
# Run this cell only if you have trained the model at least once before. Otherwise, skip it.
! /root/miniforge3/bin/python birdfsd_yolov5/model_utils/download_weights.py --model-version latest --output 'best.pt'

## Data Preprocessing

In [ ]:
! /root/miniforge3/bin/python birdfsd_yolov5/model_utils/s3_helper.py --download-dataset

In [ ]:
! tar -xf dataset-YOLO-*.tar
! mv dataset-YOLO/dataset_config.yml .
! /root/miniforge3/bin/python birdfsd_yolov5/model_utils/relative_to_abs.py

## Available Training Options

```
usage: train.py [-h] [--weights WEIGHTS] [--cfg CFG] [--data DATA] [--hyp HYP]
                [--epochs EPOCHS] [--batch-size BATCH_SIZE] [--imgsz IMGSZ]
                [--rect] [--resume [RESUME]] [--nosave] [--noval]
                [--noautoanchor] [--noplots] [--evolve [EVOLVE]]
                [--bucket BUCKET] [--cache [CACHE]] [--image-weights]
                [--device DEVICE] [--multi-scale] [--single-cls]
                [--optimizer {SGD,Adam,AdamW}] [--sync-bn] [--workers WORKERS]
                [--project PROJECT] [--name NAME] [--exist-ok] [--quad]
                [--cos-lr] [--label-smoothing LABEL_SMOOTHING]
                [--patience PATIENCE] [--freeze FREEZE [FREEZE ...]]
                [--save-period SAVE_PERIOD] [--local_rank LOCAL_RANK]
                [--entity ENTITY] [--upload_dataset [UPLOAD_DATASET]]
                [--bbox_interval BBOX_INTERVAL]
                [--artifact_alias ARTIFACT_ALIAS]

optional arguments:
  -h, --help            show this help message and exit
  --weights WEIGHTS     initial weights path
  --cfg CFG             model.yaml path
  --data DATA           dataset.yaml path
  --hyp HYP             hyperparameters path
  --epochs EPOCHS
  --batch-size BATCH_SIZE
                        total batch size for all GPUs, -1 for autobatch
  --imgsz IMGSZ, --img IMGSZ, --img-size IMGSZ
                        train, val image size (pixels)
  --rect                rectangular training
  --resume [RESUME]     resume most recent training
  --nosave              only save final checkpoint
  --noval               only validate final epoch
  --noautoanchor        disable AutoAnchor
  --noplots             save no plot files
  --evolve [EVOLVE]     evolve hyperparameters for x generations
  --bucket BUCKET       gsutil bucket
  --cache [CACHE]       --cache images in "ram" (default) or "disk"
  --image-weights       use weighted image selection for training
  --device DEVICE       cuda device, i.e. 0 or 0,1,2,3 or cpu
  --multi-scale         vary img-size +/- 50%
  --single-cls          train multi-class data as single-class
  --optimizer {SGD,Adam,AdamW}
                        optimizer
  --sync-bn             use SyncBatchNorm, only available in DDP mode
  --workers WORKERS     max dataloader workers (per RANK in DDP mode)
  --project PROJECT     save to project/name
  --name NAME           save to project/name
  --exist-ok            existing project/name ok, do not increment
  --quad                quad dataloader
  --cos-lr              cosine LR scheduler
  --label-smoothing LABEL_SMOOTHING
                        Label smoothing epsilon
  --patience PATIENCE   EarlyStopping patience (epochs without improvement)
  --freeze FREEZE [FREEZE ...]
                        Freeze layers: backbone=10, first3=0 1 2
  --save-period SAVE_PERIOD
                        Save checkpoint every x epochs (disabled if < 1)
  --local_rank LOCAL_RANK
                        DDP parameter, do not modify
  --entity ENTITY       W&B: Entity
  --upload_dataset [UPLOAD_DATASET]
                        W&B: Upload data, "val" option
  --bbox_interval BBOX_INTERVAL
                        W&B: Set bounding-box image logging interval
  --artifact_alias ARTIFACT_ALIAS
                        W&B: Version of dataset artifact to use
```

## Train

In [ ]:
EPOCHS =  100  #@param {type:"integer"}
BATCH_SIZE = 16  #@param {type:"integer"}
if Path('best.pt').exists():
    PRETRAINED_WEIGHTS = 'best.pt'  #@param {type:"string"}
else:
    PRETRAINED_WEIGHTS = 'yolov5s'  #@param {type:"string"}

Notes on resuming training:
- If your training was interrupted for any reason you may continue where you left off using the `--resume` flag.
- If your training is fully completed, you can start a new training from any model using the `--weights` flag.

In [ ]:
! /root/miniforge3/bin/python yolov5/train.py \
    --batch $BATCH_SIZE \
    --epochs $EPOCHS \
    --data 'dataset_config.yml' \
    --weights "$PRETRAINED_WEIGHTS"